# Import Necessary Packages

In [3]:
import numpy as np
import pandas as pd
import glob
import nltk
import spacy
import gensim
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Read in Syracuse University Commencement Speeches

In [4]:
path = '/Users/Ryan/Dropbox/college/7_Semester_V/IST_736/final_project/all_documents.txt'

def text_to_sentences(text):
    sentences = re.split(r"[.!?]", text)
    sentences = [sentence.strip(" ") for sentence in sentences]
    return sentences

r = open(path, "r", encoding = "UTF-8")

text = r.read()

data=text_to_sentences(text)

data

['"I am proud that a Newhouse graduate was selected student speaker, but Kaitlyn, you sure are a hard act to follow',
 'Thank you, Chancellor Syverud, for your kind words, but I need to correct one thing',
 'A giant I am not',
 'Good about morning, trustees, Deans, members of the faculty, families of the graduates, and an especially good morning to graduates',
 'I grew up in the newspaper business',
 'One of the greats of my newspaper world, the late Salsburger of the New York Times once told me a universal truth which I am happy to share with you today, and I quote, there is no such thing as a too short speech',
 "Happily for you I've taken that dictum to the heart",
 'But this occasion, which I know has special meaning for you, has great meaning for me too',
 '\nI arrived at Syracuse as a freshman in 1947',
 "And while I did not achieve what you have, it took me 69 years to get a degree, I am extremely moved to be here alongside you today and if you will indulge me, I'd like to tell 

# Tokenize Phrases

In [5]:
def tokenize(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(tokenize(data))

print(data_words) #check some of the tokens from the senteces

[['am', 'proud', 'that', 'newhouse', 'graduate', 'was', 'selected', 'student', 'speaker', 'but', 'kaitlyn', 'you', 'sure', 'are', 'hard', 'act', 'to', 'follow'], ['thank', 'you', 'chancellor', 'syverud', 'for', 'your', 'kind', 'words', 'but', 'need', 'to', 'correct', 'one', 'thing'], ['giant', 'am', 'not'], ['good', 'about', 'morning', 'trustees', 'deans', 'members', 'of', 'the', 'faculty', 'families', 'of', 'the', 'graduates', 'and', 'an', 'especially', 'good', 'morning', 'to', 'graduates'], ['grew', 'up', 'in', 'the', 'newspaper', 'business'], ['one', 'of', 'the', 'greats', 'of', 'my', 'newspaper', 'world', 'the', 'late', 'salsburger', 'of', 'the', 'new', 'york', 'times', 'once', 'told', 'me', 'universal', 'truth', 'which', 'am', 'happy', 'to', 'share', 'with', 'you', 'today', 'and', 'quote', 'there', 'is', 'no', 'such', 'thing', 'as', 'too', 'short', 'speech'], ['happily', 'for', 'you', 've', 'taken', 'that', 'dictum', 'to', 'the', 'heart'], ['but', 'this', 'occasion', 'which', 'kno

# Lemmatization

In [6]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

['be proud newhouse graduate be select student speaker kaitlyn sure be hard act follow']


# Create Document-Word Matrix

In [7]:
vectorizer = CountVectorizer(       
                             stop_words='english', #remove stop words
                             lowercase=True,#convert all words to lowercase
                             ngram_range=(1,2), #utilize uni and bigrams
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

# Build LDA model with sklearn

In [8]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=5, #set number of topics
                                      max_iter=10, #set max learning iterations
                                      learning_method='online',   
                                      random_state=0, #set the random state
                                      n_jobs=-1, #use all processors
                                      batch_size=128,
                                      evaluate_every=-1,
                                      total_samples=1000,
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None, total_samples=1000,
             verbose=0)


# View Dominant Topic per Document

In [9]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.15,0.01,0.82,0.01,0.01,2
Doc1,0.01,0.95,0.01,0.01,0.01,1
Doc2,0.60,0.10,0.10,0.10,0.10,0
Doc3,0.01,0.01,0.97,0.01,0.01,2
Doc4,0.03,0.86,0.03,0.03,0.03,1
Doc5,0.01,0.98,0.01,0.01,0.01,1
Doc6,0.90,0.03,0.03,0.03,0.03,0
Doc7,0.02,0.93,0.02,0.02,0.02,1
Doc8,0.03,0.03,0.03,0.03,0.87,4
Doc9,0.01,0.01,0.01,0.97,0.01,3


# Topic Distribution Across Documents

In [10]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,338
1,2,321
2,1,304
3,3,290
4,4,283


# Visualize the LDA model with pyLDAvis

In [11]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      22.473732        1       1  -60.164085   18.002224
1      21.414630        1       2   54.012108  110.730927
3      21.255726        1       3  -27.389553 -119.924187
4      17.472020        1       4  129.525345   -9.739214
0      17.383892        1       5  116.314049 -151.301437, topic_info=      Category       Freq            Term      Total  loglift  logprob
term                                                                  
4145   Default  54.000000            good  54.000000  30.0000  30.0000
6856   Default  40.000000             new  40.000000  29.0000  29.0000
7334   Default  80.000000          people  80.000000  28.0000  28.0000
10105  Default  36.000000           thing  36.000000  27.0000  27.0000
10266  Default  55.000000            time  55.000000  26.0000  26.0000
5814   Default  58.000000            life  58.000000  25.0000  25.0000
6703   Default  24.000000          mother  24.000000  24.0000  24.0000
8697   Default  60.000000             say  60.000000  23.0000  23.0000
2297   Default  42.000000             day  42.000000  22.0000  22.0000
10139  Default  63.000000           think  63.000000  21.0000  21.0000
5370   Default  44.000000            just  44.000000  20.0000  20.0000
10805  Default  47.000000              ve  47.000000  19.0000  19.0000
11481  Default  58.000000            year  58.000000  18.0000  18.0000
9970   Default  24.000000            tell  24.000000  17.0000  17.0000
1425   Default  18.000000          change  18.000000  16.0000  16.0000
4758   Default  35.000000            hope  35.000000  15.0000  15.0000
10074  Default  22.000000           thank  22.000000  14.0000  14.0000
6046   Default  28.000000            look  28.000000  13.0000  13.0000
8339   Default  15.000000        remember  15.000000  12.0000  12.0000
5512   Default  51.000000            know  51.000000  11.0000  11.0000
10758  Default  15.000000             use  15.000000  10.0000  10.0000
10562  Default  19.000000             try  19.000000   9.0000   9.0000
11313  Default  36.000000            work  36.000000   8.0000   8.0000
10966  Default  38.000000            want  38.000000   7.0000   7.0000
370    Default  14.000000        american  14.000000   6.0000   6.0000
3509   Default  11.000000            fall  11.000000   5.0000   5.0000
540    Default  19.000000             ask  19.000000   4.0000   4.0000
1927   Default  11.000000  congratulation  11.000000   3.0000   3.0000
3904   Default  17.000000          friend  17.000000   2.0000   2.0000
9530   Default  12.000000           story  12.000000   1.0000   1.0000
...        ...        ...             ...        ...      ...      ...
5576    Topic5   2.659853         know ve   3.192148   1.5672  -7.1861
7007    Topic5   2.636938      obligation   3.168216   1.5661  -7.1947
7827    Topic5   2.762471         private   3.358490   1.5543  -7.1482
4076    Topic5   2.366421            gift   2.905613   1.5444  -7.3030
979     Topic5   2.224827          border   2.739719   1.5415  -7.3647
5665    Topic5   2.393763             law   2.951923   1.5400  -7.2915
9606    Topic5  10.405420         student  12.920580   1.5331  -5.8220
9669    Topic5   6.689760         success   8.396641   1.5224  -6.2638
7879    Topic5   4.333176       professor   5.406453   1.5283  -6.6981
1298    Topic5   5.757371            case   7.356320   1.5045  -6.4139
3904    Topic5  12.851625          friend  17.822346   1.4226  -5.6109
11250   Topic5   8.853731            wish  11.968112   1.4482  -5.9835
10074   Topic5  15.261187           thank  22.720733   1.3517  -5.4390
4697    Topic5   6.872498            hold   9.393634   1.4371  -6.2368
5814    Topic5  32.374690            life  58.341814   1.1607  -4.6870
1780    Topic5   6.277556    commencement   8.678211   1.4258  -6.3274
1583    Topic5  10.880652           class  18.026038   1.2448  -5.7774
1